In [10]:
import pinocchio as pin
import os
from pinocchio.visualize import MeshcatVisualizer
import numpy as np
from numpy.linalg import solve,norm
# from robot_descriptions.loaders.pinocchio import load_robot_description
# import gepetto.corbaserver

In [11]:
urdf_filename = "/home/adi/hum_rob_ws/src/six_dof/urdf/6dof_from_hip.urdf"
mesh_dir = "/home/adi/hum_rob_ws/src/six_dof/meshes"

ros_package_path = "/home/adi/hum_rob_ws/src"  # Replace with your path
os.environ["ROS_PACKAGE_PATH"] = ros_package_path

In [12]:
# building model from URDF, model is an object that includes kinematic and ineratia params
model = pin.buildModelFromUrdf(urdf_filename)

visual_model = pin.buildGeomFromUrdf(
    model, urdf_filename, pin.GeometryType.VISUAL, package_dirs=mesh_dir
)  # very important

collision_model = pin.buildGeomFromUrdf(
    model, urdf_filename, pin.GeometryType.COLLISION, package_dirs=mesh_dir
)

geom_data = pin.GeometryData(collision_model)


# data is an object that holds values that are a result of computation
data = model.createData()

# print('standard model: dim=' + str(len(model.joints)))
# for jn in model.joints:
#     print(jn)
# print('-' * 30)


# # Create a list of joints to lock
# jointsToLock = ['right_foot_shin', 'right_shin_thigh', 'right_thigh_body']
 
# # Get the ID of all existing joints
# jointsToLockIDs = []
# for jn in jointsToLock:
#     if model.existJointName(jn):
#         jointsToLockIDs.append(model.getJointId(jn))
#     else:
#         print('Warning: joint ' + str(jn) + ' does not belong to the model!')

# print(jointsToLockIDs)
 
# geom_models = [visual_model, collision_model]
# model_reduced, geometric_models_reduced = pin.buildReducedModel(
#     model,
#     list_of_geom_models=geom_models,
#     list_of_joints_to_lock=jointsToLockIDs,
#     reference_configuration=pin.neutral(model))
# # geometric_models_reduced is a list, ordered as the passed variable "geom_models" so:
# visual_model_reduced, collision_model_reduced = geometric_models_reduced[
#     0], geometric_models_reduced[1]

In [13]:
visualizer = MeshcatVisualizer(model, collision_model, visual_model)


visualizer.initViewer()
visualizer.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7014/static/


In [14]:
q = pin.neutral(model)
print(q)
eps = 1e-3
DT = 1e-1
damp = 1e-1
IT_MAX = 20000
JOINT_ID = 8


R_des = np.array([[1, 0, 0], [0, 0.000796327, -1], [0, 1, 0.000796327]])
#R_des = np.eye(3)

oMdes = pin.SE3(
    R_des, np.array([0.0235,-0.030,-0.265])
)  # think of oMdes as oTdes, the transfromation from desired to origin expressed in origin
i = 0

[0. 0. 0. 0. 0. 0. 0. 0.]


In [15]:
visualizer.display(q)
visualizer.viewer.jupyter_cell()

In [16]:
while True:
    pin.forwardKinematics(model, data, q)
    
    iMd = data.oMi[JOINT_ID].actInv(oMdes) #active inverse, chnages basis and moves the vector/point to the appropriate position in the new basis
    err = pin.log(iMd).vector  # in joint frame  The logarithm map (log()) is a mathematical operation that maps elements from the group SE(3) to its associated Lie algebra, se(3). This operation effectively converts the complex rotational and translational transformation into a simpler vector representation that captures the "difference" or "error" between two poses in SE(3).
    norm_err=norm(err)
    if norm_err < eps:
        success = True
        break
    if i >= IT_MAX:
        success = False
        break
    J = pin.computeJointJacobian(
        model, data, q, JOINT_ID
    )  # jacobian computed in e-e frame
    J = -np.dot(pin.Jlog6(iMd.inverse()), J)
    q_dot = -J.T.dot(solve(J.dot(J.T) + damp * np.eye(6), err))
    q = pin.integrate(model, q, q_dot * DT)
    
    #Collision check
    pin.updateGeometryPlacements(
        model, data, collision_model, geom_data, q
    )  # updates geom_data by reference

    # an active pair is a pair of bodies in a joint considered for collision
    collision_detected = pin.computeCollisions(
        model, data, collision_model, geom_data, q, True
    )  # this returns data,geom_data. polymorphic function

    if not collision_detected:
        for idx in range(model.nq):
            # Assuming model.lowerPositionLimit and model.upperPositionLimit store the limits
            q[idx] = max(
                model.lowerPositionLimit[idx],
                min(q[idx], model.upperPositionLimit[idx]),
            )
    
        print(q)
        visualizer.display(q)
    print("*************"+str(norm_err)+"***********")
    #print(i)
    i+=1
    #time.sleep(0.05)

if success:
    print("Convergence achieved!")
else:
    print(
        "\nWarning: the iterative algorithm has not reached convergence to the desired precision"
    )

print("\nresult: %s" % q.flatten().tolist())
print("\nfinal error: %s" % err.T)



[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  4.97570193e-05 -3.20019471e-03  1.09807673e-04  2.79918461e-03]
*************0.03887406138558539***********
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  9.49158859e-05 -6.30515889e-03  2.26220054e-04  5.53261093e-03]
*************0.0381777904718926***********
[ 0.          0.          0.          0.          0.0001359  -0.00931835
  0.00034816  0.00820111]
*************0.03751216042208111***********
[ 0.          0.          0.          0.          0.00017308 -0.01224306
  0.00047465  0.01080559]
*************0.03687541765318362***********
[ 0.          0.          0.          0.          0.0002068  -0.01508239
  0.00060485  0.01334701]
*************0.03626599721180788***********
[ 0.          0.          0.          0.          0.00023739 -0.0178393
  0.00073798  0.01582637]
*************0.035682496413317645***********
[ 0.          0.          0.          0.          0.00026513 -0.0205166
  0.000

In [17]:
pin.forwardKinematics(model,data,q)
 
# Print out the placement of each joint of the kinematic tree
for name, oMi in zip(model.names, data.oMi):
    print(("{:<24} : {: .3f} {: .3f} {: .3f}"
          .format( name, *oMi.translation.T.flat )))

print(data.oMi[8])

universe                 :  0.000  0.000  0.000
body_left_thigh_lateral  :  0.100  0.000  0.000
body_left_thigh          :  0.100  0.000  0.000
left_thigh_shin          :  0.142 -0.000 -0.160
left_shin_foot           :  0.123 -0.000 -0.290
right_body_thigh_lateral :  0.000  0.000  0.000
right_body_thigh         :  0.041  0.000  0.000
right_thigh_shin         :  0.044 -0.072 -0.143
right_shin_foot          :  0.024 -0.030 -0.266
  R =
           1 -2.17923e-05  4.52008e-10
 1.73538e-08  0.000775585           -1
 2.17923e-05            1  0.000775585
  p =  0.0235058 -0.0301301  -0.265991



In [18]:
result_deg=[np.rad2deg(i) for i in q]
print(result_deg)

[0.0, 0.0, 0.0, 0.0, 0.001248607397187592, -26.681957929120298, 45.49742238661571, -18.814276049130424]
